Модуль для расчета гиперпараметров

In [ ]:
pip install lightfm


     |████████████████████████████████| 307kB 2.6MB/s 
  Created wheel for lightfm: filename=lightfm-1.15-cp36-cp36m-linux_x86_64.whl size=707611 sha256=71463099a66a2455d0a6c392c443e695c5e902e16a5bbcd26855e084fa7ae8eb
  Stored in directory: /root/.cache/pip/wheels/eb/bb/ac/188385a5da6627956be5d9663928483b36da576149ab5b8f79
Successfully built lightfm


In [ ]:
pip install scikit-optimize

     |████████████████████████████████| 81kB 2.0MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

DIR = '/content/drive/My Drive/Colab Notebooks/Data_Sceince/Module_5/recommendationsv4/'
DIR_TEST = '/content/drive/My Drive/Colab Notebooks/Data_Sceince/Module_5/TEST/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [ ]:
import numpy as np # linear algebra
import pandas as pd

import json
import scipy.sparse as sparse

from lightfm import LightFM

import sklearn
from sklearn.model_selection import train_test_split

from lightfm.data import Dataset

#Читаем файлы
data = pd.read_csv(DIR + 'train.csv',low_memory=False )
test = pd.read_csv(DIR +'test.csv',low_memory=False )


# Определяем выбросы, вероятно спам
def get_spam(field):
  med = data[field].value_counts().describe()['50%']
  std = data[field].value_counts().describe()['std']
  counts = pd.DataFrame(data[field].value_counts())
  spam = list(counts[counts[field] > med + 3 * std].index)
  return spam

item_spam = get_spam('itemid')
user_spam = get_spam('userid')
data = data[~((data['itemid'].isin(item_spam)) & (data['userid'].isin(user_spam)))]

# Упорядочим наши данные по верифицированности ответа и времени
data.sort_values(by = ['verified', 'unixReviewTime'], ascending= True, inplace = True)

#Удалим дубли, оставим только по одной паре  пользователь товар, 
#оставим последний отзыв,считая, что  могут меняться вкусы и качество товаров
data.drop_duplicates(subset =['userid','unixReviewTime','itemid'], keep = 'last',inplace = True)
data = data.reset_index()


data_all = pd.concat([data, test])
#Словарь соответствия asin и itemid
mapper = dict(zip(data_all.asin, data_all.itemid))

def get_by_key(field, dict_from, dict_to):
   if field in dict_from.keys():
       dict_to[mapper.get(dict_from.get('asin'))] = dict_from.get(field)    

with open(DIR + 'meta_Grocery_and_Gourmet_Food_small.json') as file:
     d_main_category = {}
     d_brand = {}
     d_category = {}
     for line in file.readlines(): 
         lines = json.loads(line) 
         get_by_key('brand', lines, d_brand)
         get_by_key('main_cat', lines, d_main_category)
         get_by_key('category', lines, d_category)

set_itemid = set(data['itemid']) | set(test['itemid']) 
#Подготовим словари с фичами для модели, обеспечим уникальность значения
d_brand_ml =  {x:'brand:' + y for x,y in d_brand.items()}
d_main_category_ml = {x:'main_cat:' + y for x,y in d_main_category.items()}
#Поиграем самой категорией, возьмем самый нижний уровень
d_last_category_ml = {x:'last_cat:' + y[-1] for x,y in d_category.items()}       

#Формируем  mapping для всех пользователей и товаров(train + test)

dataset = Dataset() #item_identity_features=False)
dataset.fit((x for x in pd.concat([data, test])['userid']),
            (x for x in pd.concat([data, test])['itemid']))

# Добавляем фичи по товарам 
dataset.fit_partial(items=(x for x in d_last_category_ml.keys()),
                    item_features=(x for x in d_last_category_ml.values()))

dataset.fit_partial(items=(x for x in d_main_category_ml.keys()),
                    item_features=(x for x in d_main_category_ml.values()))

dataset.fit_partial(items=(x for x in d_brand_ml.keys()),
                    item_features=(x for x in d_brand_ml.values()))

# Готовим фичи по товарам
def get_item_features(itemid):
   def add_from_dict(dict_):
      if dict_.get(itemid):
         list_feat.append(dict_.get(itemid)) 

   list_feat =[]
   add_from_dict(d_brand_ml)
   add_from_dict(d_last_category_ml)
   add_from_dict(d_main_category_ml)
   return list_feat

item_features = dataset.build_item_features(((x, get_item_features(x))
                                               for x in set_itemid ))

user_id_map, user_feature_map, item_id_map, item_feature_map = dataset.mapping()


In [ ]:
from skopt import forest_minimize

NUM_THREADS = 8 #число потоков

def objective(params):
    # unpack
    epochs, learning_rate, no_components, alpha = params
    
    user_alpha = alpha
    item_alpha = alpha
    model = LightFM(loss='logistic',
                    random_state=32,
                    learning_rate=learning_rate,
                    no_components=no_components,
                    user_alpha=user_alpha,
                    item_alpha=item_alpha)
    model.fit(weights, epochs=epochs, num_threads=NUM_THREADS
               ,item_features=item_features
              )    
    preds = model.predict(test_data.userid.apply(lambda x: user_id_map[x]).values,
                              test_data.itemid.apply(lambda x: item_id_map[x]).values 
                              ,item_features=item_features
                             )
    
    out = sklearn.metrics.roc_auc_score(test_data.rating,preds)
    print(f'epochs:{epochs}, lr:{learning_rate}, no_comp:{no_components}, alpha:{alpha}, result:{out}') 
    return 1-out
#With the objective function defined, we can define ranges for our hyperparameters. These can either be simple max and mins or we can assume a distribution like below. With the ranges defined, we simple call forest_minimize and wait a pretty long time.

#Разбиваем наш датасет на обучающую и тестовую выборки
train_data, test_data = train_test_split(data,random_state=32, shuffle=True) 

#Готовим основную матрицу
(interactions, weights) = dataset.build_interactions(((x,y,z) 
                for x,y, z in zip(train_data.userid.values,train_data.itemid.values, train_data.rating.values)))

space = [(1, 260), # epochs
         (10**-4, 1.0, 'log-uniform'), # learning_rate
         (20, 200), # no_components
         (10**-6, 10**-1, 'log-uniform'), # alpha
        ]

res_fm = forest_minimize(objective, space, n_calls=25,
                     random_state=0,
                     verbose=True)
print('Maximimum p@k found: {:6.5f}'.format(-res_fm.fun))
print('Optimal parameters:')
params = ['epochs', 'learning_rate', 'no_components', 'alpha']
for (p, x_) in zip(params, res_fm.x):
    print('{}: {}'.format(p, x_))

Iteration No: 1 started. Evaluating function at random point.
epochs:48, lr:0.07257005721594281, no_comp:87, alpha:0.019486241836466404, result:0.5704933573729551
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 161.1278
Function value obtained: 0.4295
Current minimum: 0.4295
Iteration No: 2 started. Evaluating function at random point.
epochs:10, lr:0.031207198770106446, no_comp:56, alpha:3.073781785362612e-05, result:0.6948227337970182
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 20.5202
Function value obtained: 0.3052
Current minimum: 0.3052
Iteration No: 3 started. Evaluating function at random point.
epochs:89, lr:0.001232041050356271, no_comp:59, alpha:0.0004410365667970993, result:0.643704854840476
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 193.2969
Function value obtained: 0.3563
Current minimum: 0.3052
Iteration No: 4 started. Evaluating function at random point.
epochs:89, lr:0.003725110709391402, no_comp:45, 